In [1]:
import requests
from bs4 import BeautifulSoup
import os
import time

## `資恐防制法公告制裁名單`

In [2]:
# 目標網站
base_url_1 = "https://www.aml-cft.moj.gov.tw"
target_url_1 = "https://www.aml-cft.moj.gov.tw/624184/624196/624197/?Page=1&PageSize=40&type="

In [3]:
# 設定 headers，模擬瀏覽器
headers = {"User-Agent": "Mozilla/5.0"}

# 設定Proxy
proxies = {
        'https': 'http://99999999'
}

# 發送 GET 請求
response = requests.get(target_url_1, headers=headers,proxies=proxies)
response.raise_for_status()  # 確保請求成功

# 解析 HTML
soup = BeautifulSoup(response.text, "html.parser")

# 找到 `section class="lp"` 區塊
lp_section = soup.find("section", class_="lp")

link_1 = []
# 提取符合條件的 `<a>` 標籤的 `href` 屬性    
if lp_section:
    for a_tag in lp_section.find_all("a", href=True):
        href = a_tag["href"]
        # **只保留格式類似 "/624184/624196/624197/xxxxxxx/post" 的連結**
        if "/624184/624196/624197/" in href and "/post" in href:
            full_link = base_url_1 + href  # 補全網址
            link_1.append(full_link)

In [4]:
base_directionary_1 = "U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/資恐防制法公告制裁名單"

# 迭代爬取每個網址
for url in link_1:
    print(f"🔍 正在處理: {url}")

    # 發送 GET 請求獲取 HTML
    response = requests.get(url,proxies=proxies)

    # 檢查請求是否成功
    if response.status_code != 200:
        print(f"⚠️ 無法訪問 {url}，狀態碼: {response.status_code}")
        continue

    # 解析 HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # **步驟 1：找到 h2 class="title"，作為主要資料夾名稱**
    title_element = soup.find("h2", class_="title")
    folder_name = title_element.text.strip() if title_element else "default_folder"

    # **步驟 2：找到 div class="info" 內的 <time datetime>**
    date_element = soup.find("div", class_="info").find("time", datetime=True) if soup.find("div", class_="info") else None
    publish_date = date_element["datetime"].split(" ")[0] if date_element else "0000-00-00"

    # **處理不合法的字元**
    invalid_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|']
    for char in invalid_chars:
        folder_name = folder_name.replace(char, '')

    # **步驟 3：加上日期，建立資料夾**
    save_dir = os.path.join(base_directionary_1, f"{publish_date} {folder_name}")   # 加上日期前綴
    os.makedirs(save_dir, exist_ok=True)

    print(f"📁 PDF 下載目錄: {save_dir}")

    # **步驟 4：查找 div.file_download，取得 PDF 連結**
    file_download_div = soup.find("div", class_="file_download")

    pdf_links = []
    if file_download_div:
        for a_tag in file_download_div.find_all("a", href=True):
            href = a_tag["href"]
            if ".pdf" in href.lower():  # 確保是 PDF 檔案
                full_url = "https://www.aml-cft.moj.gov.tw" + href.split("?")[0]  # 移除 URL 參數
                pdf_links.append(full_url)

    # **步驟 5：檢查是否找到 PDF**
    if not pdf_links:
        print(f"⚠️ 未找到 PDF 連結: {url}")
        continue

    # **步驟 6：下載 PDF**
    for pdf_url in pdf_links:
        pdf_name = pdf_url.split("/")[-1]
        pdf_path = os.path.join(save_dir, pdf_name)

        print(f"📥 下載中: {pdf_url} -> {pdf_path}")

        pdf_response = requests.get(pdf_url, stream=True,proxies=proxies)
        if pdf_response.status_code == 200:
            with open(pdf_path, "wb") as file:
                for chunk in pdf_response.iter_content(1024):
                    file.write(chunk)
            print(f"✅ 下載完成: {pdf_name}")
        else:
            print(f"⚠️ 下載失敗: {pdf_url}")

    # **增加延遲，避免被封鎖**
    time.sleep(2)

print("🎉 所有網址處理完成！")

🔍 正在處理: https://www.aml-cft.moj.gov.tw/624184/624196/624197/1238131/post
📁 PDF 下載目錄: U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/資恐防制法公告制裁名單\2024-12-05 聯合國安全理事會伊斯蘭國與基地組織制裁委員會（ISIL and Al-Qaida Sanctions Committee）制裁名單更新訊息
📥 下載中: https://www.aml-cft.moj.gov.tw/media/381374/附件一-新聞稿sc15916.pdf -> U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/資恐防制法公告制裁名單\2024-12-05 聯合國安全理事會伊斯蘭國與基地組織制裁委員會（ISIL and Al-Qaida Sanctions Committee）制裁名單更新訊息\附件一-新聞稿sc15916.pdf
✅ 下載完成: 附件一-新聞稿sc15916.pdf
📥 下載中: https://www.aml-cft.moj.gov.tw/media/381375/附件二-制裁名單.pdf -> U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/資恐防制法公告制裁名單\2024-12-05 聯合國安全理事會伊斯蘭國與基地組織制裁委員會（ISIL and Al-Qaida Sanctions Committee）制裁名單更新訊息\附件二-制裁名單.pdf
✅ 下載完成: 附件二-制裁名單.pdf
🔍 正在處理: https://www.aml-cft.moj.gov.tw/624184/624196/624197/1215055/post
📁 PDF 下載目錄: U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/資恐防制法公告制裁名單\2024-09-20 聯合國安全理事會第1718號制裁委員會制裁名單更新訊息
📥 下載中: https://www.aml-cft.moj.gov.tw/media/374055/法務部公告1130920-1.pdf -> U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/資恐防制法公告制裁名單\2024-09-20 聯合國安全理事會第1718號制裁委員會制裁名單更新訊息\法

## `高風險國家或地區名單`

In [11]:
# 目標網站
base_url_2 = "https://www.aml-cft.moj.gov.tw"
target_url_2 = "https://www.aml-cft.moj.gov.tw/624184/624196/624198/?Page=1&PageSize=40&type="

In [12]:
# 設定 headers，模擬瀏覽器
headers = {"User-Agent": "Mozilla/5.0"}

# 設定Proxy
proxies = {
        'https': 'http://192.168.1.152:8080'
}

# 發送 GET 請求
response = requests.get(target_url_2, headers=headers,proxies=proxies)
response.raise_for_status()  # 確保請求成功

# 解析 HTML
soup = BeautifulSoup(response.text, "html.parser")

# 找到 `section class="lp"` 區塊
lp_section = soup.find("section", class_="lp")

link_2 = []
# 提取符合條件的 `<a>` 標籤的 `href` 屬性    
if lp_section:
    for a_tag in lp_section.find_all("a", href=True):
        href = a_tag["href"]
        # **只保留格式類似 "/624184/624196/624198/xxxxxxx/post" 的連結**
        if "/624184/624196/624198/" in href and "/post" in href:
            full_link = base_url_2 + href  # 補全網址
            link_2.append(full_link)

In [13]:
base_directionary_2 = "U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/高風險國家或地區名單"

# 迭代爬取每個網址
for url in link_2:
    print(f"🔍 正在處理: {url}")

    # 發送 GET 請求獲取 HTML
    response = requests.get(url,proxies=proxies)

    # 檢查請求是否成功
    if response.status_code != 200:
        print(f"⚠️ 無法訪問 {url}，狀態碼: {response.status_code}")
        continue

    # 解析 HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # **步驟 1：找到 h2 class="title"，作為主要資料夾名稱**
    title_element = soup.find("h2", class_="title")
    folder_name = title_element.text.strip() if title_element else "default_folder"

    # **步驟 2：找到 div class="info" 內的 <time datetime>**
    date_element = soup.find("div", class_="info").find("time", datetime=True) if soup.find("div", class_="info") else None
    publish_date = date_element["datetime"].split(" ")[0] if date_element else "0000-00-00"

    # **處理不合法的字元**
    invalid_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|']
    for char in invalid_chars:
        folder_name = folder_name.replace(char, '')

    # **步驟 3：加上日期，建立資料夾**
    save_dir = os.path.join(base_directionary_2, f"{publish_date} {folder_name}")   # 加上日期前綴
    os.makedirs(save_dir, exist_ok=True)

    print(f"📁 PDF 下載目錄: {save_dir}")

    # **步驟 4：查找 div.file_download，取得 PDF 連結**
    file_download_div = soup.find("div", class_="file_download")

    pdf_links = []
    if file_download_div:
        for a_tag in file_download_div.find_all("a", href=True):
            href = a_tag["href"]
            if ".pdf" in href.lower():  # 確保是 PDF 檔案
                full_url = "https://www.aml-cft.moj.gov.tw" + href.split("?")[0]  # 移除 URL 參數
                pdf_links.append(full_url)

    # **步驟 5：檢查是否找到 PDF**
    if not pdf_links:
        print(f"⚠️ 未找到 PDF 連結: {url}")
        continue

    # **步驟 6：下載 PDF**
    for pdf_url in pdf_links:
        pdf_name = pdf_url.split("/")[-1]
        pdf_path = os.path.join(save_dir, pdf_name)

        print(f"📥 下載中: {pdf_url} -> {pdf_path}")

        pdf_response = requests.get(pdf_url, stream=True,proxies=proxies)
        if pdf_response.status_code == 200:
            with open(pdf_path, "wb") as file:
                for chunk in pdf_response.iter_content(1024):
                    file.write(chunk)
            print(f"✅ 下載完成: {pdf_name}")
        else:
            print(f"⚠️ 下載失敗: {pdf_url}")

    # **增加延遲，避免被封鎖**
    time.sleep(2)

print("🎉 所有網址處理完成！")

🔍 正在處理: https://www.aml-cft.moj.gov.tw/624184/624196/624198/1227474/post
📁 PDF 下載目錄: U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/高風險國家或地區名單\2024-11-05 「防制洗錢金融行動工作組織」公布高風險及加強監督國家或地區名單相關訊息
📥 下載中: https://www.aml-cft.moj.gov.tw/media/377829/法務部公告1131105.pdf -> U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/高風險國家或地區名單\2024-11-05 「防制洗錢金融行動工作組織」公布高風險及加強監督國家或地區名單相關訊息\法務部公告1131105.pdf
✅ 下載完成: 法務部公告1131105.pdf
📥 下載中: https://www.aml-cft.moj.gov.tw/media/377830/附件1.pdf -> U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/高風險國家或地區名單\2024-11-05 「防制洗錢金融行動工作組織」公布高風險及加強監督國家或地區名單相關訊息\附件1.pdf
✅ 下載完成: 附件1.pdf
📥 下載中: https://www.aml-cft.moj.gov.tw/media/377831/附件2.pdf -> U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/高風險國家或地區名單\2024-11-05 「防制洗錢金融行動工作組織」公布高風險及加強監督國家或地區名單相關訊息\附件2.pdf
✅ 下載完成: 附件2.pdf
🔍 正在處理: https://www.aml-cft.moj.gov.tw/624184/624196/624198/1191585/post
📁 PDF 下載目錄: U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/高風險國家或地區名單\2024-07-05 「防制洗錢金融行動工作組織」公布高風險及加強監督國家或地區名單相關訊息
📥 下載中: https://www.aml-cft.moj.gov.tw/media/364645/法務部公告1130705.pdf -> U:/【全權委託組】/intern/江彥文/機構法人洗錢防

## `聯合國安理會入港禁令船舶清單及交通部航港局關注船舶清單`

In [15]:
# 目標網站
base_url_3 = "https://www.aml-cft.moj.gov.tw"
target_url_3 = "https://www.aml-cft.moj.gov.tw/624184/624196/815341/?Page=1&PageSize=20&type="

In [16]:
# 設定 headers，模擬瀏覽器
headers = {"User-Agent": "Mozilla/5.0"}

# 設定Proxy
proxies = {
        'https': 'http://192.168.1.152:8080'
}

# 發送 GET 請求
response = requests.get(target_url_3, headers=headers,proxies=proxies)
response.raise_for_status()  # 確保請求成功

# 解析 HTML
soup = BeautifulSoup(response.text, "html.parser")

# 找到 `section class="lp"` 區塊
lp_section = soup.find("section", class_="lp")

link_3 = []
# 提取符合條件的 `<a>` 標籤的 `href` 屬性    
if lp_section:
    for a_tag in lp_section.find_all("a", href=True):
        href = a_tag["href"]
        # **只保留格式類似 "/624184/624196/815341/xxxxxxx/post" 的連結**
        if "/624184/624196/815341/" in href and "/post" in href:
            full_link = base_url_3 + href  # 補全網址
            link_3.append(full_link)

In [17]:
base_directionary_3 = "U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/聯合國安理會入港禁令船舶清單及交通部航港局關注船舶清單"

# 迭代爬取每個網址
for url in link_3:
    print(f"🔍 正在處理: {url}")

    # 發送 GET 請求獲取 HTML
    response = requests.get(url,proxies=proxies)

    # 檢查請求是否成功
    if response.status_code != 200:
        print(f"⚠️ 無法訪問 {url}，狀態碼: {response.status_code}")
        continue

    # 解析 HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # **步驟 1：找到 h2 class="title"，作為主要資料夾名稱**
    title_element = soup.find("h2", class_="title")
    folder_name = title_element.text.strip() if title_element else "default_folder"

    # **步驟 2：找到 div class="info" 內的 <time datetime>**
    date_element = soup.find("div", class_="info").find("time", datetime=True) if soup.find("div", class_="info") else None
    publish_date = date_element["datetime"].split(" ")[0] if date_element else "0000-00-00"

    # **處理不合法的字元**
    invalid_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|']
    for char in invalid_chars:
        folder_name = folder_name.replace(char, '')

    # **步驟 3：加上日期，建立資料夾**
    save_dir = os.path.join(base_directionary_3, f"{publish_date} {folder_name}")   # 加上日期前綴
    os.makedirs(save_dir, exist_ok=True)

    print(f"📁 PDF 下載目錄: {save_dir}")

    # **步驟 4：查找 div.file_download，取得 PDF 連結**
    file_download_div = soup.find("div", class_="file_download")

    pdf_links = []
    if file_download_div:
        for a_tag in file_download_div.find_all("a", href=True):
            href = a_tag["href"]
            if ".pdf" in href.lower():  # 確保是 PDF 檔案
                full_url = "https://www.aml-cft.moj.gov.tw" + href.split("?")[0]  # 移除 URL 參數
                pdf_links.append(full_url)

    # **步驟 5：檢查是否找到 PDF**
    if not pdf_links:
        print(f"⚠️ 未找到 PDF 連結: {url}")
        continue

    # **步驟 6：下載 PDF**
    for pdf_url in pdf_links:
        pdf_name = pdf_url.split("/")[-1]
        pdf_path = os.path.join(save_dir, pdf_name)

        print(f"📥 下載中: {pdf_url} -> {pdf_path}")

        pdf_response = requests.get(pdf_url, stream=True,proxies=proxies)
        if pdf_response.status_code == 200:
            with open(pdf_path, "wb") as file:
                for chunk in pdf_response.iter_content(1024):
                    file.write(chunk)
            print(f"✅ 下載完成: {pdf_name}")
        else:
            print(f"⚠️ 下載失敗: {pdf_url}")

    # **增加延遲，避免被封鎖**
    time.sleep(2)

print("🎉 所有網址處理完成！")

🔍 正在處理: https://www.aml-cft.moj.gov.tw/624184/624196/815341/980742/post
📁 PDF 下載目錄: U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/聯合國安理會入港禁令船舶清單及交通部航港局關注船舶清單\2022-08-04 轉知涉及國安疑慮之禁止進港船舶清單
📥 下載中: https://www.aml-cft.moj.gov.tw/media/289410/1110727交通部航港局來文.pdf -> U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/聯合國安理會入港禁令船舶清單及交通部航港局關注船舶清單\2022-08-04 轉知涉及國安疑慮之禁止進港船舶清單\1110727交通部航港局來文.pdf
✅ 下載完成: 1110727交通部航港局來文.pdf
🔍 正在處理: https://www.aml-cft.moj.gov.tw/624184/624196/815341/971468/post
📁 PDF 下載目錄: U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/聯合國安理會入港禁令船舶清單及交通部航港局關注船舶清單\2022-07-05 轉知聯合國安理會入港禁令船舶清單及交通部航港局關注船舶清單
📥 下載中: https://www.aml-cft.moj.gov.tw/media/286103/1110704航港局來文.pdf -> U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/聯合國安理會入港禁令船舶清單及交通部航港局關注船舶清單\2022-07-05 轉知聯合國安理會入港禁令船舶清單及交通部航港局關注船舶清單\1110704航港局來文.pdf
✅ 下載完成: 1110704航港局來文.pdf
📥 下載中: https://www.aml-cft.moj.gov.tw/media/286105/聯合國安全理事會北韓制裁委員會公告受制裁船舶清單.pdf -> U:/【全權委託組】/intern/江彥文/機構法人洗錢防制/聯合國安理會入港禁令船舶清單及交通部航港局關注船舶清單\2022-07-05 轉知聯合國安理會入港禁令船舶清單及交通部航港局關注船舶清單\聯合國安全理事會北韓制裁委員會公告受制裁船舶清單.pdf
✅ 下載完成: 聯